In [17]:
import pandas as pd
import tensorflow as tf
import tempfile
import numpy as np

from sklearn.cross_validation import train_test_split

data = pd.read_csv('data/b33e77f31fe04969803d8c93eee90701_merge.csv')

data = data[data['cur_cpd'] > 0]

data.drop(['cur_time','cur_cpd',], axis = 1, inplace = True)
data.head(10)
#data.dtypes

,kid,cur_hour,cur_rank1_rate,cur_price,cur_profit
255,40177047550,22,0.67,0.55,1.51
256,40177047550,23,0.67,0.55,3.02
279,40177047550,14,0.00,0.50,3.16
284,40177047550,16,0.00,0.70,1.80
286,40177047550,17,0.00,0.74,1.75
294,40177047550,21,1.00,1.00,0.96
296,40177047550,22,0.80,1.00,1.17
297,40177047550,22,0.80,1.00,2.34
298,40177047550,23,0.60,0.90,1.70
316,40177047550,2,0.00,1.00,1.77


In [27]:
train_data, test_data = train_test_split(data, train_size = 0.8, random_state = 100)

COLUMNS = ['kid','cur_hour', 'cur_rank1_rate','cur_price','cur_profit']
CONTINUOUS_COLUMNS = ['cur_rank1_rate','cur_price','cur_hour']
CATEGORICAL_COLUMNS = ['kid',]

LABEL_COLUMN = 'cur_profit'

def input_fn(df):
  continuous_cols = {k: tf.constant(df[k].values)
                     for k in CONTINUOUS_COLUMNS}

  categorical_cols = {k: tf.SparseTensor(
      indices=[[i, 0] for i in range(df[k].size)],
      values=df[k].values,
      shape=[df[k].size, 1])
                      for k in CATEGORICAL_COLUMNS}
    
  label = tf.constant(df[LABEL_COLUMN].values)

  feature_cols = dict(continuous_cols)
  feature_cols.update(categorical_cols)

  return feature_cols, label

def train_input_fn():
  return input_fn(train_data)

def eval_input_fn():
  return input_fn(test_data)

In [28]:
kid = tf.contrib.layers.sparse_column_with_hash_bucket("kid", hash_bucket_size=100000)

hour = tf.contrib.layers.real_valued_column("cur_hour")
rank1_rate = tf.contrib.layers.real_valued_column("cur_rank1_rate")
price = tf.contrib.layers.real_valued_column("cur_price")

model_dir = tempfile.mkdtemp()
m = tf.contrib.learn.LinearRegressor(feature_columns=[
  hour, rank1_rate, price])

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.


In [29]:
m.fit(input_fn=train_input_fn, steps=200)

LinearRegressor()

In [30]:
results = m.evaluate(input_fn=eval_input_fn, steps=1)
print(results)


{'global_step': 200, 'loss': 32.179867}
